In [1]:
# .env 파일을 읽어서 환경변수로 설정
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()


True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Runnable")


LangSmith 추적을 시작합니다.
[프로젝트명]
Runnable


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# chain 을 생성합니다.
chain = prompt | llm


In [4]:
# chain 을 실행합니다.
chain.invoke({"num": 5})


AIMessage(content='5의 10배는 50입니다.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'stop', 'logprobs': None}, id='run-1fe6e070-4bee-4d9f-b861-c7f954952efa-0', usage_metadata={'input_tokens': 14, 'output_tokens': 10, 'total_tokens': 24})

In [5]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num": 10})


{'num': 10}

In [6]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경되었습니다.
runnable_chain.invoke(10)

AIMessage(content='100입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-689fcfd0-0ea0-4c2e-9624-edd3a96dd3bb-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19})

In [13]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

# RunnableParallel 인스턴스를 생성합니다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

In [14]:
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [11]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)


In [16]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")


{'capital': AIMessage(content='서울특별시입니다.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 19, 'total_tokens': 29}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9c60d589-7dcf-4090-8f64-8554e083bf70-0', usage_metadata={'input_tokens': 19, 'output_tokens': 10, 'total_tokens': 29}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,363km² 입니다.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 20, 'total_tokens': 45}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-34fb0310-c0b2-469e-bbd8-2811ff5b8a31-0', usage_metadata={'input_tokens': 20, 'output_tokens': 25, 'total_tokens': 45})}

In [18]:
def combined_text(text):
  return text['a'].content + text['b'].content

In [24]:
model = ChatOpenAI(model_name="gpt-4o-mini")

In [25]:
combined_chain | {"info": RunnableLambda(combined_text)} | PromptTemplate.from_template("{info}") | model

{
  capital: {
             country: RunnablePassthrough()
           }
           | PromptTemplate(input_variables=['country'], template='{country} 의 수도는?')
           | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11d7bd690>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11d804450>, openai_api_key=SecretStr('**********'), openai_proxy=''),
  area: {
          country: RunnablePassthrough()
        }
        | PromptTemplate(input_variables=['country'], template='{country} 의 면적은?')
        | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11d867650>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11d838110>, openai_api_key=SecretStr('**********'), openai_proxy='')
}
| {
    info: RunnableLambda(combined_text)
  }
| PromptTemplate(input_variables=['info'], template='{info}')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11d7bf8d0>, a

In [26]:
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 19, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3ac953a3-1272-4657-95be-1b5948127ead-0', usage_metadata={'input_tokens': 19, 'output_tokens': 15, 'total_tokens': 34}),
 'area': AIMessage(content='대한민국의 면적은 약 100,363 제곱 킬로미터 입니다.', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 20, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5df6eb26-5480-4323-bc17-2c6bd023eb75-0', usage_metadata={'input_tokens': 20, 'output_tokens': 28, 'total_tokens': 48})}